In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from delta.tables import DeltaTable

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "canais"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_canal"
TABELA_INVALIDOS_DESTINO = "tb_canal_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"
nome_tabela_invalidos = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_INVALIDOS_DESTINO}"

In [0]:
# StringType foi importado no código original, mas não estava na sua lista de imports

# Leitura da Bronze
canais_bronze_df = spark.read.table(nome_tabela_origem)

# --- INÍCIO DA LÓGICA DE FILTRO POR CARGA MAIS RECENTE ---
# 1. Encontrar a data/timestamp de ingestão mais recente
max_dt_ingestion = (
    canais_bronze_df
    .agg(F.max(F.col("ingestion_timestamp")).alias("max_ts"))
    .first()["max_ts"]
)

# 2. Filtrar o DataFrame Bronze apenas pela carga mais recente
df_filtrado = canais_bronze_df.filter(
    F.col("ingestion_timestamp") == F.lit(max_dt_ingestion)
)
# --- FIM DA LÓGICA DE FILTRO POR CARGA MAIS RECENTE ---


# Variações de status para mapeamento
ativos_variacoes = ["ativo", "ativoo", "atvo", "atvivo", "atvio", "ativ", "atv", "at1vo", "a tivo"]
inativos_variacoes = ["inativo", "inatvo", "invativo", "inativoo", "inativ", "inativ0", "in ativo", "inat1vo", "in at1vo"]

# Transformação: cd_canal gerado por hash do nome
df_transformado = (
    # O filtro já foi aplicado em df_filtrado
    df_filtrado
    .select(
        F.abs(F.hash(F.lower(F.trim(F.col("nome_canal"))))).cast("bigint").alias("cd_canal"),
        F.upper(F.trim(F.col("nome_canal"))).alias("nm_canal"),
        F.col("canal_status").cast(StringType()).alias("st_ativo_raw"),
        F.col("ingestion_timestamp").alias("dt_ingestion"),
        F.col("origem").alias("dc_origem")
    )
    .withColumn(
        "st_ativo",
        F.when(
            F.lower(F.col("st_ativo_raw")).isin([v.lower() for v in inativos_variacoes]),
            F.lit(False)
        ).when(
            F.lower(F.col("st_ativo_raw")).isin([v.lower() for v in ativos_variacoes]),
            F.lit(True)
        ).otherwise(F.lit(None))
    )
    .dropDuplicates(["cd_canal"])
)

# Validação com flags granulares
df_validacao = (
    df_transformado
    .withColumn("flag_nome_valido", F.col("nm_canal").isNotNull() & (F.trim(F.col("nm_canal")) != ""))
    .withColumn("flag_status_valido", F.col("st_ativo").isNotNull())
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_nome_valido") & F.col("flag_status_valido"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

# Separar válidos e inválidos
df_validos = df_validacao.filter(F.col("flag_qualidade") == "OK")
df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

# DataFrame final para Silver (remover flags de auditoria)
df_silver = df_validos.select(
    "cd_canal",
    "nm_canal",
    "st_ativo",
    "dt_ingestion",
    "dc_origem"
)

In [0]:
delta_table = DeltaTable.forName(spark, nome_tabela_destino)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.cd_canal = origem.cd_canal"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {nome_tabela_destino} atualizada com sucesso!")

df_invalidos.write.format("delta").mode("overwrite").saveAsTable(nome_tabela_invalidos)

print(f"✅ Tabela {nome_tabela_invalidos} atualizada para auditoria!")